This notebook was part of [Lesson 7](https://course.fast.ai/videos/?lesson=7) of the Practical Deep Learning for Coders course.

# Predicting English word version of numbers using an RNN

We were using RNNs as part of our language model in the previous lesson.  Today, we will dive into more details of what RNNs are and how they work.  We will do this using the problem of trying to predict the English word version of numbers.

Let's predict what should come next in this sequence:

*eight thousand one , eight thousand two , eight thousand three , eight thousand four , eight thousand five , eight thousand six , eight thousand seven , eight thousand eight , eight thousand nine , eight thousand ten , eight thousand eleven , eight thousand twelve...*


Jeremy created this synthetic dataset to have a better way to check if things are working, to debug, and to understand what was going on. When experimenting with new ideas, it can be nice to have a smaller dataset to do so, to quickly get a sense of whether your ideas are promising (for other examples, see [Imagenette and Imagewoof](https://github.com/fastai/imagenette)) This English word numbers will serve as a good dataset for learning about RNNs.  Our task today will be to predict which word comes next when counting.

### In deep learning, there are 2 types of numbers

**Parameters** are numbers that are learned.  **Activations** are numbers that are calculated (by affine functions & element-wise non-linearities).

When you learn about any new concept in deep learning, ask yourself: is this a parameter or an activation?

Note to self: Point out the hidden state, going from the version without a for-loop to the for loop.  This is the step where people get confused.

## 0. Download, explore and prepare the data

In [1]:
from fastai.text import *

In [2]:
bs=64

In [3]:
path = untar_data(URLs.HUMAN_NUMBERS)
path.ls()

[WindowsPath('C:/Users/cross-entropy/.fastai/data/human_numbers/models'),
 WindowsPath('C:/Users/cross-entropy/.fastai/data/human_numbers/train.txt'),
 WindowsPath('C:/Users/cross-entropy/.fastai/data/human_numbers/valid.txt')]

In [4]:
def readnums(file): 
    return [', '.join(o.strip() for o in open(path/file).readlines())]

#### train.txt is a sequence of numbers from 1 to 8000 written out as English words:

In [5]:
train_txt = readnums('train.txt')
print(train_txt[0][:80])
print(train_txt[0][-80:])
print(len(train_txt[0]))

one, two, three, four, five, six, seven, eight, nine, ten, eleven, twelve, thirt
even thousand nine hundred ninety eight, seven thousand nine hundred ninety nine
280597


#### valid.txt is a sequence of numbers from 8001 to 9999 written out as English words:

In [6]:
valid_txt = readnums('valid.txt')
print(valid_txt[0][0:80])
print(valid_txt[0][-80:])
print(len(valid_txt[0]))

eight thousand one, eight thousand two, eight thousand three, eight thousand fou
 nine thousand nine hundred ninety eight, nine thousand nine hundred ninety nine
74882


#### make a databunch; batch size of 64 lines of length bptt = 70 tokens per line

In [7]:
train = TextList(train_txt, path=path)
valid = TextList(valid_txt, path=path)

src = ItemLists(path=path, train=train, valid=valid).label_for_lm()
data = src.databunch(bs=bs)

In [8]:
train[0].text[:80]

'xxbos one , two , three , four , five , six , seven , eight , nine , ten , eleve'

In [9]:
len(data.valid_ds[0][0].data)

13017

In [10]:
len(data.train_ds[0][0].data)

50079

`bptt` stands for *back-propagation through time*.  This tells us how many steps of history we are considering.

In [11]:
data.bptt, len(data.valid_dl)

(70, 3)

In [12]:
len(data.train_dl)

12

We have 3 batches in our validation set:

13017 tokens, with about ~70 tokens in about a line of text, and 64 lines of text per batch.

In [13]:
13017/70/bs

2.905580357142857

In [14]:
50079/70/bs

11.178348214285714

We will store each batch in a separate variable, so we can walk through this to understand better what the RNN does at each step:

In [15]:
it = iter(data.valid_dl)
x1,y1 = next(it)
x2,y2 = next(it)
x3,y3 = next(it)
it.close()

In [16]:
x1

tensor([[ 2, 19, 11,  ..., 36,  9, 19],
        [ 9, 19, 11,  ..., 24, 20,  9],
        [11, 27, 18,  ...,  9, 19, 11],
        ...,
        [20, 11, 20,  ..., 11, 20, 10],
        [20, 11, 20,  ..., 24,  9, 20],
        [20, 10, 26,  ..., 20, 11, 20]], device='cuda:0')

`numel()` is a [PyTorch method](https://pytorch.org/docs/stable/torch.html#torch.numel) to return the number of elements in a tensor:

In [17]:
x1.numel()+x2.numel()+x3.numel()

13440

In [18]:
print(x1.numel())
print(x2.numel())
print(x3.numel())

4480
4480
4480


In [19]:
x1.shape, y1.shape

(torch.Size([64, 70]), torch.Size([64, 70]))

In [20]:
x2.shape, y2.shape

(torch.Size([64, 70]), torch.Size([64, 70]))

In [21]:
x3.shape, y3.shape

(torch.Size([64, 70]), torch.Size([64, 70]))

#### vocabulary has 40 tokens

In [22]:
v = data.valid_ds.vocab

In [23]:
print(len(v.itos))
v.itos

40


['xxunk',
 'xxpad',
 'xxbos',
 'xxeos',
 'xxfld',
 'xxmaj',
 'xxup',
 'xxrep',
 'xxwrep',
 ',',
 'hundred',
 'thousand',
 'one',
 'two',
 'three',
 'four',
 'five',
 'six',
 'seven',
 'eight',
 'nine',
 'twenty',
 'thirty',
 'forty',
 'fifty',
 'sixty',
 'seventy',
 'eighty',
 'ninety',
 'ten',
 'eleven',
 'twelve',
 'thirteen',
 'fourteen',
 'fifteen',
 'sixteen',
 'seventeen',
 'eighteen',
 'nineteen',
 'xxfake']

In [24]:
x1[0,:]

tensor([ 2, 19, 11, 12,  9, 19, 11, 13,  9, 19, 11, 14,  9, 19, 11, 15,  9, 19,
        11, 16,  9, 19, 11, 17,  9, 19, 11, 18,  9, 19, 11, 19,  9, 19, 11, 20,
         9, 19, 11, 29,  9, 19, 11, 30,  9, 19, 11, 31,  9, 19, 11, 32,  9, 19,
        11, 33,  9, 19, 11, 34,  9, 19, 11, 35,  9, 19, 11, 36,  9, 19],
       device='cuda:0')

In [25]:
y1[0,:]

tensor([19, 11, 12,  9, 19, 11, 13,  9, 19, 11, 14,  9, 19, 11, 15,  9, 19, 11,
        16,  9, 19, 11, 17,  9, 19, 11, 18,  9, 19, 11, 19,  9, 19, 11, 20,  9,
        19, 11, 29,  9, 19, 11, 30,  9, 19, 11, 31,  9, 19, 11, 32,  9, 19, 11,
        33,  9, 19, 11, 34,  9, 19, 11, 35,  9, 19, 11, 36,  9, 19, 11],
       device='cuda:0')

In [26]:
v.itos[19], v.itos[11], v.itos[12], v.itos[9], v.itos[19], v.itos[11],v.itos[13],v.itos[9]

('eight', 'thousand', 'one', ',', 'eight', 'thousand', 'two', ',')

In [27]:
v.textify(x1[0])

'xxbos eight thousand one , eight thousand two , eight thousand three , eight thousand four , eight thousand five , eight thousand six , eight thousand seven , eight thousand eight , eight thousand nine , eight thousand ten , eight thousand eleven , eight thousand twelve , eight thousand thirteen , eight thousand fourteen , eight thousand fifteen , eight thousand sixteen , eight thousand seventeen , eight'

In [28]:
v.textify(y1[0])

'eight thousand one , eight thousand two , eight thousand three , eight thousand four , eight thousand five , eight thousand six , eight thousand seven , eight thousand eight , eight thousand nine , eight thousand ten , eight thousand eleven , eight thousand twelve , eight thousand thirteen , eight thousand fourteen , eight thousand fifteen , eight thousand sixteen , eight thousand seventeen , eight thousand'

In [29]:
v.textify(x1[1,:])

', eight thousand forty six , eight thousand forty seven , eight thousand forty eight , eight thousand forty nine , eight thousand fifty , eight thousand fifty one , eight thousand fifty two , eight thousand fifty three , eight thousand fifty four , eight thousand fifty five , eight thousand fifty six , eight thousand fifty seven , eight thousand fifty eight , eight thousand fifty nine ,'

In [30]:
v.textify(y1[1,:])

'eight thousand forty six , eight thousand forty seven , eight thousand forty eight , eight thousand forty nine , eight thousand fifty , eight thousand fifty one , eight thousand fifty two , eight thousand fifty three , eight thousand fifty four , eight thousand fifty five , eight thousand fifty six , eight thousand fifty seven , eight thousand fifty eight , eight thousand fifty nine , eight'

In [31]:
v.textify(x2[0])

'thousand eighteen , eight thousand nineteen , eight thousand twenty , eight thousand twenty one , eight thousand twenty two , eight thousand twenty three , eight thousand twenty four , eight thousand twenty five , eight thousand twenty six , eight thousand twenty seven , eight thousand twenty eight , eight thousand twenty nine , eight thousand thirty , eight thousand thirty one , eight thousand thirty two ,'

In [32]:
v.textify(x2[1])

'eight thousand sixty , eight thousand sixty one , eight thousand sixty two , eight thousand sixty three , eight thousand sixty four , eight thousand sixty five , eight thousand sixty six , eight thousand sixty seven , eight thousand sixty eight , eight thousand sixty nine , eight thousand seventy , eight thousand seventy one , eight thousand seventy two , eight thousand seventy three , eight thousand'

In [33]:
v.textify(x3[0])

'eight thousand thirty three , eight thousand thirty four , eight thousand thirty five , eight thousand thirty six , eight thousand thirty seven , eight thousand thirty eight , eight thousand thirty nine , eight thousand forty , eight thousand forty one , eight thousand forty two , eight thousand forty three , eight thousand forty four , eight thousand forty five , eight thousand forty six , eight'

In [34]:
v.textify(x1[1])

', eight thousand forty six , eight thousand forty seven , eight thousand forty eight , eight thousand forty nine , eight thousand fifty , eight thousand fifty one , eight thousand fifty two , eight thousand fifty three , eight thousand fifty four , eight thousand fifty five , eight thousand fifty six , eight thousand fifty seven , eight thousand fifty eight , eight thousand fifty nine ,'

In [35]:
v.textify(x2[1])

'eight thousand sixty , eight thousand sixty one , eight thousand sixty two , eight thousand sixty three , eight thousand sixty four , eight thousand sixty five , eight thousand sixty six , eight thousand sixty seven , eight thousand sixty eight , eight thousand sixty nine , eight thousand seventy , eight thousand seventy one , eight thousand seventy two , eight thousand seventy three , eight thousand'

In [36]:
v.textify(x3[1])

'seventy four , eight thousand seventy five , eight thousand seventy six , eight thousand seventy seven , eight thousand seventy eight , eight thousand seventy nine , eight thousand eighty , eight thousand eighty one , eight thousand eighty two , eight thousand eighty three , eight thousand eighty four , eight thousand eighty five , eight thousand eighty six , eight thousand eighty seven , eight thousand eighty'

In [37]:
v.textify(x3[-1])

'ninety , nine thousand nine hundred ninety one , nine thousand nine hundred ninety two , nine thousand nine hundred ninety three , nine thousand nine hundred ninety four , nine thousand nine hundred ninety five , nine thousand nine hundred ninety six , nine thousand nine hundred ninety seven , nine thousand nine hundred ninety eight , nine thousand nine hundred ninety nine xxbos eight thousand one , eight'

In [38]:
data.show_batch(ds_type=DatasetType.Valid)

idx,text
0,"thousand forty seven , eight thousand forty eight , eight thousand forty nine , eight thousand fifty , eight thousand fifty one , eight thousand fifty two , eight thousand fifty three , eight thousand fifty four , eight thousand fifty five , eight thousand fifty six , eight thousand fifty seven , eight thousand fifty eight , eight thousand fifty nine , eight thousand sixty , eight thousand sixty"
1,"eight , eight thousand eighty nine , eight thousand ninety , eight thousand ninety one , eight thousand ninety two , eight thousand ninety three , eight thousand ninety four , eight thousand ninety five , eight thousand ninety six , eight thousand ninety seven , eight thousand ninety eight , eight thousand ninety nine , eight thousand one hundred , eight thousand one hundred one , eight thousand one"
2,"thousand one hundred twenty four , eight thousand one hundred twenty five , eight thousand one hundred twenty six , eight thousand one hundred twenty seven , eight thousand one hundred twenty eight , eight thousand one hundred twenty nine , eight thousand one hundred thirty , eight thousand one hundred thirty one , eight thousand one hundred thirty two , eight thousand one hundred thirty three , eight thousand"
3,"three , eight thousand one hundred fifty four , eight thousand one hundred fifty five , eight thousand one hundred fifty six , eight thousand one hundred fifty seven , eight thousand one hundred fifty eight , eight thousand one hundred fifty nine , eight thousand one hundred sixty , eight thousand one hundred sixty one , eight thousand one hundred sixty two , eight thousand one hundred sixty three"
4,"thousand one hundred eighty three , eight thousand one hundred eighty four , eight thousand one hundred eighty five , eight thousand one hundred eighty six , eight thousand one hundred eighty seven , eight thousand one hundred eighty eight , eight thousand one hundred eighty nine , eight thousand one hundred ninety , eight thousand one hundred ninety one , eight thousand one hundred ninety two , eight thousand"


We will iteratively consider a few different models, building up to a more traditional RNN.

## 1. Single fully connected model: predict the next word

#### make a databunch; batch size 64 lines of bptt = 3 tokens  per line

In [39]:
data = src.databunch(bs=bs, bptt=3)

In [40]:
print(len(data.train_dl))
print(len(data.valid_dl))

261
68


In [41]:
bs

64

In [42]:
x,y = data.one_batch()
x.shape,y.shape

(torch.Size([64, 3]), torch.Size([64, 3]))

#### number of tokens in vocabulary, i.e. length of vocabulary vector

In [43]:
nv = len(v.itos)
nv

40

#### size of hidden layer

In [44]:
nh=64

In [45]:
def loss4(input,target): return F.cross_entropy(input, target[:,-1])
def acc4 (input,target): return accuracy(input, target[:,-1])

#### first token in each line of the batch

In [46]:
x[:,0]

tensor([13, 13, 10,  9, 18,  9, 11, 11, 13, 19, 16, 23, 24,  9, 12,  9, 13, 14,
        15, 11, 10, 22, 15,  9, 10, 14, 11, 16, 10, 28, 11,  9, 20,  9, 15, 15,
        11, 18, 10, 28, 23, 24,  9, 16, 10, 16, 19, 20, 12, 10, 22, 16, 17, 17,
        17, 11, 24, 10,  9, 15, 16,  9, 18, 11])

Layer names:
- `i_h`: input to hidden
- `h_h`: hidden to hidden
- `h_o`: hidden to output
- `bn`: batchnorm

In [47]:
class Model0(nn.Module):
    def __init__(self):
        super().__init__()
        self.i_h = nn.Embedding(nv,nh)  # green arrow
        self.h_h = nn.Linear(nh,nh)     # brown arrow
        self.h_o = nn.Linear(nh,nv)     # blue arrow
        self.bn = nn.BatchNorm1d(nh)
        
    def forward(self, x):
        h = self.bn(F.relu(self.i_h(x[:,0])))
        if x.shape[1]>1:
            h = h + self.i_h(x[:,1])
            h = self.bn(F.relu(self.h_h(h)))
        if x.shape[1]>2:
            h = h + self.i_h(x[:,2])
            h = self.bn(F.relu(self.h_h(h)))
        return self.h_o(h)

In [48]:
learn = Learner(data, Model0(), loss_func=loss4, metrics=acc4)

In [49]:
learn.fit_one_cycle(6, 1e-4)

epoch,train_loss,valid_loss,acc4,time
0,3.666404,3.578313,0.066406,00:04
1,2.725338,2.835792,0.409237,00:02
2,2.120784,2.373993,0.449678,00:02
3,1.895006,2.194904,0.462316,00:02
4,1.814307,2.139137,0.459099,00:02
5,1.797665,2.131413,0.459559,00:02


## Same thing with a loop

Let's refactor this to use a for-loop.  This does the same thing as before:

In [50]:
class Model1(nn.Module):
    def __init__(self):
        super().__init__()
        self.i_h = nn.Embedding(nv,nh)  # green arrow
        self.h_h = nn.Linear(nh,nh)     # brown arrow
        self.h_o = nn.Linear(nh,nv)     # blue arrow
        self.bn = nn.BatchNorm1d(nh)
        
    def forward(self, x):
        h = torch.zeros(x.shape[0], nh).to(device=x.device)
        for i in range(x.shape[1]):
            h = h + self.i_h(x[:,i])
            h = self.bn(F.relu(self.h_h(h)))
        return self.h_o(h)

This is the difference between unrolled (what we had before) and rolled (what we have now) RNN diagrams:

In [51]:
learn = Learner(data, Model1(), loss_func=loss4, metrics=acc4)

In [52]:
learn.fit_one_cycle(6, 1e-4)

epoch,train_loss,valid_loss,acc4,time
0,3.644308,3.542901,0.093290,00:02
1,2.744738,2.723261,0.420037,00:02
2,2.126453,2.276868,0.454044,00:02
3,1.893005,2.130797,0.465533,00:02
4,1.814050,2.089912,0.465993,00:02
5,1.797763,2.084440,0.466452,00:02


Our accuracy is about the same, since we are doing the same thing as before.

## 2. Multi fully connected model: ie. predict an arbitrary number of next words

Before, we were just predicting the last word in a line of text.  Given 70 tokens, what is token 71?  That approach was throwing away a lot of data.  Why not predict token 2 from token 1, then predict token 3, then predict token 4, and so on?  We will modify our model to do this.

In [53]:
data = src.databunch(bs=bs, bptt=20)

In [54]:
x,y = data.one_batch()
x.shape,y.shape

(torch.Size([64, 20]), torch.Size([64, 20]))

In [55]:
class Model2(nn.Module):
    def __init__(self):
        super().__init__()
        self.i_h = nn.Embedding(nv,nh)
        self.h_h = nn.Linear(nh,nh)
        self.h_o = nn.Linear(nh,nv)
        self.bn = nn.BatchNorm1d(nh)
        
    def forward(self, x):
        h = torch.zeros(x.shape[0], nh).to(device=x.device)
        res = []
        for i in range(x.shape[1]):
            h = h + self.i_h(x[:,i])
            h = F.relu(self.h_h(h))
            res.append(self.h_o(self.bn(h)))
        return torch.stack(res, dim=1)

In [56]:
learn = Learner(data, Model2(), metrics=accuracy)

In [57]:
learn.fit_one_cycle(10, 1e-4, pct_start=0.1)

epoch,train_loss,valid_loss,accuracy,time
0,3.817364,3.735988,0.045313,00:01
1,3.648061,3.536991,0.101705,00:01
2,3.442540,3.360008,0.181534,00:01
3,3.239639,3.216457,0.210369,00:01
4,3.062086,3.109120,0.243253,00:01
5,2.918954,3.033288,0.256250,00:01
6,2.811532,2.984315,0.266832,00:01
7,2.737720,2.957824,0.272372,00:01
8,2.692700,2.947629,0.275284,00:01
9,2.669264,2.946105,0.275568,00:01


In [58]:
learn.fit_one_cycle??


Note that our accuracy is worse now, because we are doing a harder task.  When we predict word k (k<70), we have less history to help us then when we were only predicting word 71.

## 3. Maintain state with an RNN

To address this issue, let's keep the hidden state from the previous line of text, so we are not starting over again on each new line of text.

In [59]:
class Model3(nn.Module):
    def __init__(self):
        super().__init__()
        self.i_h = nn.Embedding(nv,nh)
        self.h_h = nn.Linear(nh,nh)
        self.h_o = nn.Linear(nh,nv)
        self.bn = nn.BatchNorm1d(nh)
        self.h = torch.zeros(bs, nh).cuda()
        
    def forward(self, x):
        res = []
        h = self.h
        for i in range(x.shape[1]):
            h = h + self.i_h(x[:,i])
            h = F.relu(self.h_h(h))
            res.append(self.bn(h))
        self.h = h.detach()
        res = torch.stack(res, dim=1)
        res = self.h_o(res)
        return res

In [60]:
learn = Learner(data, Model3(), metrics=accuracy)

In [61]:
learn.fit_one_cycle(20, 3e-3)

epoch,train_loss,valid_loss,accuracy,time
0,3.666807,3.620840,0.067259,00:01
1,3.102550,2.725251,0.388707,00:01
2,2.325210,2.141467,0.375142,00:01
3,1.862248,2.082507,0.316903,00:01
4,1.641223,2.058397,0.317685,00:01
5,1.489763,1.863659,0.482955,00:01
6,1.319308,1.726680,0.520028,00:01
7,1.147587,1.651183,0.569176,00:01
8,0.997068,1.666286,0.562926,00:01
9,0.872947,1.660153,0.588423,00:01


Now we are getting greater accuracy than before!

## Use PyTorch's nn.RNN

Let's refactor the above to use PyTorch's RNN.  This is what you would use in practice, but now you know the inside details!

Question: why is PyTorch's RNN so much better than ours?

In [62]:
class Model4(nn.Module):
    def __init__(self):
        super().__init__()
        self.i_h = nn.Embedding(nv,nh)
        self.rnn = nn.RNN(nh,nh, batch_first=True)
        self.h_o = nn.Linear(nh,nv)
        self.bn = BatchNorm1dFlat(nh)
        self.h = torch.zeros(1, bs, nh).cuda()
        
    def forward(self, x):
        res,h = self.rnn(self.i_h(x), self.h)
        self.h = h.detach()
        return self.h_o(self.bn(res))

In [63]:
learn = Learner(data, Model4(), metrics=accuracy)

In [64]:
learn.fit_one_cycle(20, 3e-3)

epoch,train_loss,valid_loss,accuracy,time
0,3.465608,3.087324,0.329474,00:00
1,2.745341,2.140465,0.460156,00:00
2,2.100147,2.004082,0.331747,00:00
3,1.755720,2.070867,0.315625,00:00
4,1.588078,2.048706,0.319531,00:00
5,1.467861,1.847858,0.470597,00:00
6,1.312439,1.873531,0.480966,00:00
7,1.138188,1.734502,0.499432,00:00
8,0.999427,1.620505,0.510369,00:00
9,0.872186,1.627890,0.540270,00:00


## 4. 2-layer GRU gives a dramatic boost to accuracy!

When you have long time scales and deeper networks, these become impossible to train.  One way to address this is to add mini-NN to decide how much of the green arrow and how much of the orange arrow to keep.  These mini-NNs can be GRUs or LSTMs.  We will cover more details of this in a later lesson.

In [65]:
class Model5(nn.Module):
    def __init__(self):
        super().__init__()
        self.i_h = nn.Embedding(nv,nh)
        self.rnn = nn.GRU(nh, nh, 2, batch_first=True)
        self.h_o = nn.Linear(nh,nv)
        self.bn = BatchNorm1dFlat(nh)
        self.h = torch.zeros(2, bs, nh).cuda()
        
    def forward(self, x):
        res,h = self.rnn(self.i_h(x), self.h)
        self.h = h.detach()
        return self.h_o(self.bn(res))

In [66]:
learn = Learner(data, Model5(), metrics=accuracy)

In [67]:
learn.fit_one_cycle(10, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,2.630658,2.207643,0.421733,00:00
1,1.542026,1.284782,0.692401,00:00
2,0.745779,0.864366,0.805114,00:00
3,0.365024,0.978731,0.818608,00:00
4,0.186690,1.120214,0.812500,00:00
5,0.100699,1.122629,0.804332,00:00
6,0.057756,1.121250,0.817969,00:00
7,0.035486,1.128709,0.823793,00:00
8,0.023646,1.218956,0.819531,00:00
9,0.017180,1.190759,0.817614,00:00


## Let's make our own GRU

### Using PyTorch's GRUCell

Axis 0 is the batch dimension, and axis 1 is the time dimension.  We want to loop through axis 1:

In [68]:
def rnn_loop(cell, h, x):
    res = []
    for x_ in x.transpose(0,1):
        h = cell(x_, h)
        res.append(h)
    return torch.stack(res, dim=1)

In [69]:
class Model6(Model5):
    def __init__(self):
        super().__init__()
        self.rnnc = nn.GRUCell(nh, nh)
        self.h = torch.zeros(bs, nh).cuda()
        
    def forward(self, x):
        res = rnn_loop(self.rnnc, self.h, self.i_h(x))
        self.h = res[:,-1].detach()
        return self.h_o(self.bn(res))

In [70]:
learn = Learner(data, Model6(), metrics=accuracy)
learn.fit_one_cycle(10, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,2.822955,2.410875,0.440767,00:01
1,1.915055,1.828685,0.476065,00:01
2,1.448396,1.932703,0.554901,00:01
3,0.953431,1.479870,0.698580,00:01
4,0.539017,1.346834,0.758026,00:01
5,0.292759,1.182884,0.797443,00:01
6,0.163847,1.232905,0.779901,00:01
7,0.098245,1.259229,0.795668,00:01
8,0.064760,1.308113,0.784162,00:01
9,0.047976,1.287600,0.787926,00:01


### With a custom (non-PyTorch) GRUCell

The following is based on code from [emadRad](https://github.com/emadRad/lstm-gru-pytorch/blob/master/lstm_gru.ipynb):

In [71]:
class GRUCell(nn.Module):
    def __init__(self, ni, nh):
        super(GRUCell, self).__init__()
        # super().__init__()
        
        self.ni,self.nh = ni,nh
        self.i2h = nn.Linear(ni, 3*nh)
        self.h2h = nn.Linear(nh, 3*nh)
    
    def forward(self, x, h):
        gate_x = self.i2h(x).squeeze()
        gate_h = self.h2h(h).squeeze()
        i_r,i_u,i_n = gate_x.chunk(3, 1)
        h_r,h_u,h_n = gate_h.chunk(3, 1)
        
        resetgate = torch.sigmoid(i_r + h_r)
        updategate = torch.sigmoid(i_u + h_u)
        newgate = torch.tanh(i_n + (resetgate*h_n))
        return updategate*h + (1-updategate)*newgate

In [72]:
class Model7(Model6):
    def __init__(self):
        super().__init__()
        self.rnnc = GRUCell(nh,nh)

In [73]:
learn = Learner(data, Model7(), metrics=accuracy)
learn.fit_one_cycle(10, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,2.821176,2.354398,0.383452,00:02
1,1.917785,2.276164,0.319957,00:02
2,1.471152,1.769334,0.472514,00:02
3,0.912579,1.945928,0.601634,00:02
4,0.488974,1.545381,0.660866,00:02
5,0.260052,1.455830,0.696875,00:02
6,0.143346,1.464046,0.698935,00:02
7,0.085095,1.538583,0.693324,00:02
8,0.055418,1.536705,0.694602,00:02
9,0.040662,1.586214,0.691903,00:02


In [74]:
dir(learn)

['__annotations__',
 '__class__',
 '__dataclass_fields__',
 '__dataclass_params__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__post_init__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_test_writeable_path',
 'add_time',
 'apply_dropout',
 'backward',
 'bn_wd',
 'callback_fns',
 'callbacks',
 'clip_grad',
 'create_opt',
 'data',
 'destroy',
 'dl',
 'export',
 'fit',
 'fit_fc',
 'fit_one_cycle',
 'freeze',
 'freeze_to',
 'get_preds',
 'init',
 'interpret',
 'layer_groups',
 'load',
 'loss_func',
 'lr_find',
 'lr_range',
 'metrics',
 'mixup',
 'model',
 'model_dir',
 'one_cycle_scheduler',
 'opt',
 'opt_func',
 'path',
 'pred_batch',
 'predict',
 'predict_with_mc_dropout',
 'purge',
 'recorder',
 'save',
 'show_result

In [76]:
x.shape

torch.Size([64, 20])

In [87]:
x.numpy()

array([[ 9, 30,  9, 31, ...,  9, 38,  9, 21],
       [10, 34,  9, 13, ..., 10, 38,  9, 13],
       [20,  9, 14, 10, ..., 14, 10, 27, 14],
       [10, 23, 18,  9, ..., 24,  9, 16, 10],
       ...,
       [11, 16, 10, 22, ..., 10, 23,  9, 18],
       [17, 10, 24, 15, ..., 24, 17,  9, 18],
       [10, 26,  9, 18, ...,  9, 18, 11, 18],
       [27, 17,  9, 18, ...,  9, 18, 11, 19]], dtype=int64)

In [90]:
data

TextLMDataBunch;

Train: LabelList (1 items)
x: LMTextList
xxbos one , two , three , four , five , six , seven , eight , nine , ten , eleven , twelve , thirteen , fourteen , fifteen , sixteen , seventeen , eighteen , nineteen , twenty , twenty one , twenty two , twenty three , twenty four , twenty five , twenty six , twenty seven , twenty eight , twenty nine , thirty , thirty one , thirty two , thirty three , thirty four , thirty five , thirty six , thirty seven , thirty eight , thirty nine , forty , forty one , forty two , forty three , forty four , forty five , forty six , forty seven , forty eight , forty nine , fifty , fifty one , fifty two , fifty three , fifty four , fifty five , fifty six , fifty seven , fifty eight , fifty nine , sixty , sixty one , sixty two , sixty three , sixty four , sixty five , sixty six , sixty seven , sixty eight , sixty nine , seventy , seventy one , seventy two , seventy three , seventy four , seventy five , seventy six , seventy seven , seventy eight

In [104]:
data.x[0].data

array([ 2, 12,  9, 13, ..., 20, 10, 28, 20], dtype=int64)

In [110]:
dir(data.x[0].data)

['T',
 '__abs__',
 '__add__',
 '__and__',
 '__array__',
 '__array_finalize__',
 '__array_function__',
 '__array_interface__',
 '__array_prepare__',
 '__array_priority__',
 '__array_struct__',
 '__array_ufunc__',
 '__array_wrap__',
 '__bool__',
 '__class__',
 '__complex__',
 '__contains__',
 '__copy__',
 '__deepcopy__',
 '__delattr__',
 '__delitem__',
 '__dir__',
 '__divmod__',
 '__doc__',
 '__eq__',
 '__float__',
 '__floordiv__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__hash__',
 '__iadd__',
 '__iand__',
 '__ifloordiv__',
 '__ilshift__',
 '__imatmul__',
 '__imod__',
 '__imul__',
 '__index__',
 '__init__',
 '__init_subclass__',
 '__int__',
 '__invert__',
 '__ior__',
 '__ipow__',
 '__irshift__',
 '__isub__',
 '__iter__',
 '__itruediv__',
 '__ixor__',
 '__le__',
 '__len__',
 '__lshift__',
 '__lt__',
 '__matmul__',
 '__mod__',
 '__mul__',
 '__ne__',
 '__neg__',
 '__new__',
 '__or__',
 '__pos__',
 '__pow__',
 '__radd__',
 '__rand__',
 '__rdivmod__',
 '__

In [111]:
type(data.x[0].data)

numpy.ndarray

In [112]:
type(data)

fastai.text.data.TextLMDataBunch

In [113]:
type(data.x)

fastai.text.data.LMTextList

In [130]:
type(data.x[0])

fastai.text.data.Text

In [128]:
data.x[0].data

array([ 2, 12,  9, 13, ..., 20, 10, 28, 20], dtype=int64)

In [129]:
type(data.x[0].data)

numpy.ndarray

In [131]:
dir(data.x[0].data)

['T',
 '__abs__',
 '__add__',
 '__and__',
 '__array__',
 '__array_finalize__',
 '__array_function__',
 '__array_interface__',
 '__array_prepare__',
 '__array_priority__',
 '__array_struct__',
 '__array_ufunc__',
 '__array_wrap__',
 '__bool__',
 '__class__',
 '__complex__',
 '__contains__',
 '__copy__',
 '__deepcopy__',
 '__delattr__',
 '__delitem__',
 '__dir__',
 '__divmod__',
 '__doc__',
 '__eq__',
 '__float__',
 '__floordiv__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__hash__',
 '__iadd__',
 '__iand__',
 '__ifloordiv__',
 '__ilshift__',
 '__imatmul__',
 '__imod__',
 '__imul__',
 '__index__',
 '__init__',
 '__init_subclass__',
 '__int__',
 '__invert__',
 '__ior__',
 '__ipow__',
 '__irshift__',
 '__isub__',
 '__iter__',
 '__itruediv__',
 '__ixor__',
 '__le__',
 '__len__',
 '__lshift__',
 '__lt__',
 '__matmul__',
 '__mod__',
 '__mul__',
 '__ne__',
 '__neg__',
 '__new__',
 '__or__',
 '__pos__',
 '__pow__',
 '__radd__',
 '__rand__',
 '__rdivmod__',
 '__

In [132]:
data.x.items

array([[ 2, 12,  9, 13, ..., 20, 10, 28, 20]], dtype=int64)

In [133]:
data.x.items.shape

(1, 50079)

In [108]:
dir(ItemBase)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 'apply_tfms',
 'show']

In [106]:
learn.predict??


In [140]:
type(data.x[0].data)

numpy.ndarray

In [141]:
data.x[0].data

array([ 2, 12,  9, 13, ..., 20, 10, 28, 20], dtype=int64)

In [142]:
learn.predict(data.x[0].data)

IndexError: Dimension out of range (expected to be in range of [-1, 0], but got 1)

### Connection to ULMFit

In the previous lesson, we were essentially swapping out `self.h_o` with a classifier in order to do classification on text.

RNNs are just a refactored, fully-connected neural network.

You can use the same approach for any sequence labeling task (part of speech, classifying whether material is sensitive,..)


## fin